In [1]:
## Preloads
using Statistics
using FFTW
using Plots
using BenchmarkTools
using Profile
using LinearAlgebra
using Measures
using HDF5
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using MLDatasets
using Images
theme(:juno)
using DSP
using Interpolations
using Distributed
using ProgressMeter

  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:176
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:179
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:181
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:184
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:186
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:189
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:203
  likely near /n/home12/saydjari/.julia/packages/Plots

In [2]:
addprocs(30);

In [3]:
using Random
Random.seed!(100);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

In [4]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_train = Array{Any}(undef, 0)
for i = 1:60000
    push!(lst_train,train_x[:,:,i])
end


lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

In [5]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_RR_wd2.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_train_RR_wd2.h5", "main/angles", train_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:12:53


In [6]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:53


In [7]:
Random.seed!(56);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_0.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_0.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:53


In [8]:
Random.seed!(5968);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_1.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_1.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:53


In [9]:
Random.seed!(4729);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_2.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_2.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:52


In [10]:
Random.seed!(154);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_3.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_3.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:52


In [11]:
Random.seed!(985);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_4.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_4.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:53


In [12]:
Random.seed!(46);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_5.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_5.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:53


In [13]:
Random.seed!(22);
test_angles = 2π*rand(10000);
train_angles = 2π*rand(60000);

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256),Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, zip(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_RR_wd2_6.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_RR_wd2_6.h5", "main/angles", test_angles)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:53
